## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [84]:
import pandas as pd
import numpy as np

In [85]:
df=pd.read_csv('train.csv')

In [86]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [87]:
df.shape

(20800, 5)

In [129]:
# df.isnull().sum()

In [130]:
###Drop Nan Values
df=df.dropna()

In [131]:
df.shape

(18285, 5)

In [133]:
## Get the Dependent features
y=df['label']

In [134]:
## Get the Independent Features
X=df.drop('label',axis=1)

In [135]:
X.shape

(18285, 4)

In [136]:
y.shape

(18285,)

In [137]:
import tensorflow as tf

In [138]:
tf.__version__

'2.15.0'

In [139]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [140]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [141]:
messages=X.copy()

In [142]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [153]:
messages.reset_index(inplace=True)

In [155]:
messages[1:10]

,index,id,title,author,text
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [156]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [157]:
import nltk
import re
from nltk.corpus import stopwords

In [158]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [159]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(messages['title'][i])
    review = re.sub('[^a-zA-Z]',' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
Susan Rice, Ex-Obama Adviser, Is Back in Political Cross Hairs Over Surveillance - The New York Times
The Fix Is In: NBC Affiliate Accidentally Posts Election Results A Week Early: Hillary Wins Presidency 42% to Trump’s 40%
Original unreleased transcript of Sonia Gandhi’s interview with Rajdeep Sardesai obtained
Indian Man Accused in Multimillion-Dollar Call-Center Swindle Is Held - The New York Times
7 Teenage Deaths, but No Answers for Aboriginal Canadians - The New York Times
Fashioning Cast-Iron Pans for Today’s Cooks - The New York Times
Summer Sweat: Hollywood Braces for Possible Worst Box Office in Decade
Trump Executive Orders Target Trade Abuses Amid NAFTA Intrigue - Breitbart
New York Times’s Moscow Bureau Was Targeted by Hackers - The New York Times
Barbra Streisand Says Sexism Cost Her Multiple Oscar Nominations
Comey Investigation a Red Herring to Cover Up Wikileaks Dump
Gorsuch Tries to Put Himself Above Politics in Confi

In [160]:
# corpus

In [161]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [162]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
# onehot_repr

In [163]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [164]:
onehot_repr[1]

[1532, 4404, 953, 892, 631, 4936, 963]

### Embedding Representation

In [165]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[3450 2238  499 ...    0    0    0]
 [1532 4404  953 ...    0    0    0]
 [2422  452 3467 ...    0    0    0]
 ...
 [4848 2198 4273 ...    0    0    0]
 [ 502 3108 4817 ...    0    0    0]
 [3070  353 1171 ...    0    0    0]]


In [166]:
embedded_docs[1]

array([1532, 4404,  953,  892,  631, 4936,  963,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [167]:
# embedded_docs[0]

In [168]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [169]:
len(embedded_docs),y.shape

(18285, (18285,))

In [170]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [171]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [172]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [173]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 19s 72ms/step - loss: 0.3128 - accuracy: 0.8415 - val_loss: 0.2051 - val_accuracy: 0.9109
Epoch 2/10
192/192 [==============================] - 4s 19ms/step - loss: 0.1344 - accuracy: 0.9491 - val_loss: 0.2030 - val_accuracy: 0.9216
Epoch 3/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0863 - accuracy: 0.9692 - val_loss: 0.2317 - val_accuracy: 0.9185
Epoch 4/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0585 - accuracy: 0.9802 - val_loss: 0.3267 - val_accuracy: 0.9142
Epoch 5/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0413 - accuracy: 0.9873 - val_loss: 0.3003 - val_accuracy: 0.9097
Epoch 6/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0256 - accuracy: 0.9920 - val_loss: 0.3802 - val_accuracy: 0.9094
Epoch 7/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0257 - accuracy: 0.9917 - val_loss: 0.3670 - val_accuracy: 0.907

### Adding Dropout

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [174]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [175]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [176]:
from sklearn.metrics import confusion_matrix

In [177]:
confusion_matrix(y_test,y_pred)

array([[3155,  264],
       [ 362, 2254]])

In [178]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8962717481358741

In [179]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91      3419
           1       0.90      0.86      0.88      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.89      0.89      6035
weighted avg       0.90      0.90      0.90      6035

